In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
%matplotlib inline
os.chdir('/home/ankushraut/Downloads/Datalord')

In [61]:
train = pd.read_csv('train/train.csv')
test = pd.read_csv('test/test.csv')
campaign_train = pd.read_csv('train/campaign_data.csv')
submission = pd.read_csv('sample_submission.csv')

In [62]:
train.head()

,id,user_id,campaign_id,send_date,is_open,is_click
0,42_14051,14051,42,01-09-2017 19:55,0,0
1,52_134438,134438,52,02-11-2017 12:53,0,0
2,33_181789,181789,33,24-07-2017 15:15,0,0
3,44_231448,231448,44,05-09-2017 11:36,0,0
4,29_185580,185580,29,01-07-2017 18:01,0,0


In [63]:
train.is_open.value_counts()

0    920401
1    102790
Name: is_open, dtype: int64

In [64]:
train.is_click.value_counts()

0    1010409
1      12782
Name: is_click, dtype: int64

In [65]:
train = pd.merge(train, campaign_train, on='campaign_id').sort_values('id').reset_index(drop=True)
test = pd.merge(test, campaign_train, on='campaign_id').sort_values('id').reset_index(drop = True)

In [66]:
submission = submission.sort_values('id').reset_index(drop = True)

In [67]:
test.head()

,id,campaign_id,user_id,send_date,communication_type,total_links,no_of_internal_links,no_of_images,no_of_sections,email_body,subject,email_url
0,55_100000,55,100000,09-01-2018 15:35,Corporate,11,7,5,1,Do you know? You can identify and hire the bes...,Hire Top Data Scientists In Less Than 24 Hours,http://r.newsletters.analyticsvidhya.com/7w2se...
1,55_100092,55,100092,09-01-2018 15:35,Corporate,11,7,5,1,Do you know? You can identify and hire the bes...,Hire Top Data Scientists In Less Than 24 Hours,http://r.newsletters.analyticsvidhya.com/7w2se...
2,55_10010,55,10010,09-01-2018 15:34,Corporate,11,7,5,1,Do you know? You can identify and hire the bes...,Hire Top Data Scientists In Less Than 24 Hours,http://r.newsletters.analyticsvidhya.com/7w2se...
3,55_100106,55,100106,09-01-2018 15:33,Corporate,11,7,5,1,Do you know? You can identify and hire the bes...,Hire Top Data Scientists In Less Than 24 Hours,http://r.newsletters.analyticsvidhya.com/7w2se...
4,55_100199,55,100199,09-01-2018 15:34,Corporate,11,7,5,1,Do you know? You can identify and hire the bes...,Hire Top Data Scientists In Less Than 24 Hours,http://r.newsletters.analyticsvidhya.com/7w2se...


In [68]:
submission.head()

,id,is_click
0,55_100000,0
1,55_100092,0
2,55_10010,0
3,55_100106,0
4,55_100199,0


In [69]:
train.send_date = pd.to_datetime(train.send_date)
train['minute'] = train.send_date.dt.minute
train['hour'] = train.send_date.dt.hour
train['day'] = train.send_date.dt.day
train['week'] = train.send_date.dt.week
train['month'] = train.send_date.dt.month
train['year'] = train.send_date.dt.year

test.send_date = pd.to_datetime(test.send_date)
test['minute'] = test.send_date.dt.minute
test['hour'] = test.send_date.dt.hour
test['day'] = test.send_date.dt.day
test['week'] = test.send_date.dt.week
test['month'] = test.send_date.dt.month
test['year'] = test.send_date.dt.year

In [70]:
train.communication_type.value_counts()

Newsletter         451053
Conference         332633
Upcoming Events    148964
Others              50936
Hackathon           33265
Webinar              3487
Corporate            2853
Name: communication_type, dtype: int64

In [71]:
train = train.join(pd.get_dummies(train.communication_type))
train = train.join(pd.get_dummies(train.email_body))
train = train.join(pd.get_dummies(train.subject))

test = test.join(pd.get_dummies(test.communication_type))
test = test.join(pd.get_dummies(test.email_body))
test = test.join(pd.get_dummies(test.subject))

In [72]:
test = test.reindex(columns = train.columns, fill_value=0)

In [73]:
pd.set_option('display.max_columns', None)

In [75]:
training = train.drop(labels = ['id', 'send_date', 'communication_type', 'email_body', 'subject', 'email_url', 'is_open', 'is_click'], axis = 1)
target = train['is_click']
testing = test.drop(labels = ['id', 'send_date', 'communication_type', 'email_body', 'subject', 'email_url', 'is_open', 'is_click'], axis = 1)

In [76]:
print(len(training.columns), len(testing.columns))

70 70


In [77]:
from sklearn.model_selection import train_test_split

In [78]:
xdata, xtest, ydata, ytest = train_test_split(training, target, test_size = 0.1, random_state = 1, stratify = target)
xdata = xdata.reset_index(drop = True)
xtest = xtest.reset_index(drop = True)
ydata = ydata.reset_index(drop = True)
ytest = ytest.reset_index(drop = True)

In [79]:
xtrain, xval, ytrain, yval = train_test_split(xdata, ydata, test_size = 0.25, random_state = 1, stratify = ydata)
xtrain = xtrain.reset_index(drop = True)
xval = xval.reset_index(drop = True)
ytrain = ytrain.reset_index(drop = True)
yval = yval.reset_index(drop = True)

In [80]:
import xgboost as xgb

/home/ankushraut/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [1]:
params = {"objective":"binary:logistic",
          "booster":"gbtree",
          "eta":0.1,
          "max_depth": 3,
         "subsample":0.95,
         "silent":0,
         "seed":1,
         "lambda":.01,
         "alpha":.01,
         "eval_metric":"auc"}

num_boost_round = 1000

dtrain = xgb.DMatrix(xtrain, ytrain)
dvalid = xgb.DMatrix(xval, yval)

watchlist = [(dtrain, 'train'), (dvalid, 'eval')]

booster = xgb.train(params, dtrain, num_boost_round, evals = watchlist, early_stopping_rounds = 25)

NameError: name 'xgb' is not defined

In [82]:
xtrain.describe()

,user_id,campaign_id,total_links,no_of_internal_links,no_of_images,no_of_sections,minute,hour,day,week,month,year,Conference,Corporate,Hackathon,Newsletter,Others,Upcoming Events,Webinar,"Hi ? Before I dive into why you should attend this Data Science conference, let me announce that conferences are designed - To make you think. To spark inspiration. To take you where the next big thing truly lies. DataHack Summit 2017 is one such opportunity which will take a leap into the future of Data Science. If youre on the fence about attending DataHack Summit 2017 - Conference on Artificial Intelligence & Machine Learning, here are a few reasons to register today, and start planning your trip to Bengaluru.","August Newsletter Dear AVians, We have had one of the most anticipated events of the month - Fireside Chat with DJ Patil, Former Data Scientist of the USA and I am thrilled to congratulate the whole team for putting up an excellent session as well as to those who were present there and made full out of this opportunity. If you've missed it, here's a coverage of the event for you all: Watch here. He's touched upon various important areas which I personally recommend you all should listen to! And moving to our biggest event of the year - DataHack Summit 2017. On great demands, we've extended the 'early bird tickets' offer till 7th August. So, make sure you have your booking before it finishes. As we are now finalising the Workshops and sessions of the Summit (Take a look at the sessions here) - if you have a strong command over Analytics/ Data Science specific subjects or if you know someone who does, we humbly invite you to share the dais with other experts and be one of the speakers at the Summit. You can submit your interests here. Meanwhile, upgrade your skills with an excellent Beginner's guide on Tableau, D3.js, Intro. on Genetic Algorithm and many more useful topics been explained through our articles published in July. So keep up with the pace, keep pouring your suggestions for the event as well as for our daily efforts to ensure we deliver what you are looking for! Have a great month ahead! Regards, Kunal Jain","CodeFest by IIT BHU [Coding Festival] 22 September - 24 September 2017 Dear AVian, We are announcing the fourth edition of the highly anticipated coding festival by The Department of Computer Science and Engineering, IIT(BHU) Varanasi, Codefest, that will be held from 22nd - 24th September 2017. This edition will mark an annual programming extravaganza as a dream weekend for coding enthusiasts across the world. A look back at Codefest'16 - 13,257 unique visitors 10,435 participants 2,024 institutions 92 countries INR 450,000 Prize Money CodeFest Enigma [Machine Learning] 22 September - 24 September 2017 About CodeFest: Enigma This is a machine learning event during 22 - 24 September 2017. Cash Prizes: Top 10 rankers of the hackathon will be awarded Rank 1: ?INR 20K Rank 2: ?INR 12K Rank 3: INR 8K Rank 4-10: INR 1K Rank 1 in IIT (BHU) Varanasi: INR 3K This is not all, if you get in the top 20, you will get exclusive benefits on India's Largest Data Science Event, in Bengaluru, DataHack Summit 2017. Register CodeFest Linguipedia [Natural Language Processing] 23 September - 24 September 2017 About CodeFest: Linguipedia This is a Natural Language Processing event during 23 - 24 September 2017. Cash Prizes: Top 10 rankers of the hackathon will be awarded Rank 1: ?INR 20K Rank 2: ?INR 12K Rank 3: INR 8K Rank 4-10: INR 1K Rank 1 in IIT (BHU) Varanasi: INR 3K This is not all, if you get in the top 20, you will get exclusive benefits on India's Largest Data Science Event, in Bengaluru, DataHack Summit 2017.","Hosted Your Corporate Hackathon Yet? Do You Know??? 9 out of 10 Fortune 500 companies in the past five decades are gone. The few companies which survived have only one thing in common - INNOVATION Today, innovation is the key driver for businesses to survive and grow. The best companies crowd-source innovation through h